In [1]:
import importlib
import sys
import os
import numpy as np
import time
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import librosa

import torch
from torch import nn
import torch.optim as optim

from torch.utils import data
from torchvision import transforms

sys.path.append(os.path.join(os.getcwd(), ".."))

from distiller import apputils
import ai8x

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

kws20 = importlib.import_module("datasets.kws20-horsecough")

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,multilabel_confusion_matrix

def rescale(audio, min_val=-1,max_val=1):
            sig = audio
            mean = np.average(sig)

            sig = sig-mean # REMOVE DC COMPONENT

            sig_max = np.max(sig)
            sig_min = np.min(sig)

            if sig_max >= np.abs(sig_min):
                sig_scaled = sig/sig_max
            else:
                sig_scaled = sig/np.abs(sig_min)

            return sig_scaled

In [2]:
def count_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print(params)
    return params

ai8x.set_device(device=85, simulate=False, round_avg=False)

model_name = 'equine_1'
checkpoint_dir = '../notebooks/checkpoints/'+model_name+'/'
checkpoint_file = checkpoint_dir+'/qat_'+model_name+'_best.pth.tar'

# GET MODEL STRUCTURE
mod = importlib.import_module("models.ai85net-equine")
model = mod.AI85EQUINE(num_classes=2, num_channels=128, dimensions=(128, 1), bias=False)

# LOAD MODEL CHECKPOINT
model, compression_scheduler, optimizer, start_epoch = apputils.load_checkpoint(model,checkpoint_file,lean_checkpoint=True)

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    cpu = False
else:
     device = torch.device('cpu')
     cpu = True

print('Running on device: {}'.format(torch.cuda.get_device_name()))

model = model.to(device)
model.eval()

Configuring device: MAX78000, simulate=False.
Running on device: NVIDIA GeForce RTX 2070 SUPER


AI85EQUINE(
  (drop): Dropout(p=0.2, inplace=False)
  (voice_conv1): FusedConv1dReLU(
    (activate): ReLU(inplace=True)
    (op): Conv1d(128, 100, kernel_size=(1,), stride=(1,), bias=False)
    (calc_out_shift): OutputShiftSqueeze()
    (calc_weight_scale): One()
    (scale): Scaler()
    (calc_out_scale): OutputScale()
    (quantize_weight): Empty()
    (quantize_bias): Empty()
    (clamp_weight): Empty()
    (clamp_bias): Empty()
    (quantize): Empty()
    (clamp): Clamp()
    (quantize_pool): Empty()
    (clamp_pool): Empty()
  )
  (voice_conv2): FusedConv1dReLU(
    (activate): ReLU(inplace=True)
    (op): Conv1d(100, 96, kernel_size=(3,), stride=(1,), bias=False)
    (calc_out_shift): OutputShiftSqueeze()
    (calc_weight_scale): One()
    (scale): Scaler()
    (calc_out_scale): OutputScale()
    (quantize_weight): Empty()
    (quantize_bias): Empty()
    (clamp_weight): Empty()
    (clamp_bias): Empty()
    (quantize): Empty()
    (clamp): Clamp()
    (quantize_pool): Empty()
 

In [3]:
# class_paths = {'combined': "../data/KWS_EQUINE/raw/combined/",
#                'human_cough': "C:/Users/J_C/Desktop/DATASETS_N/human_cough_v3/"}

# class_paths = {'combinedKWS': "../data/KWS_EQUINE/raw/__combinedkws/",
#                'horse_cough': "../data/KWS_EQUINE/inference/horse_cough/"}

# class_paths = {'combined': "../data/KWS_EQUINE/raw/combined/",
#                'horse_cough': "../data/KWS_EQUINE/inference/horse_cough/"}

class_paths = {'horse_cough_internet': "../data/KWS_EQUINE/inference/horse_cough/",
               'horse_cough_stable': "C:/Users/J_C/Desktop/DATASETS_N/horse_cough_stable/"}


y_true_train = []
y_pred_train = []

classes = list(class_paths.keys())
time_start = time.time()

with torch.no_grad():
    for class_ix,inf_path in enumerate(list(class_paths.values())):
        files = os.listdir(inf_path)
        file_count = len(files)
        inferences = []
        for counter,f in enumerate(files):
            try:
                print('Class: ',class_ix,'  Remaining: ',(file_count-counter))

                # CONVERT EACH AUDIO FILE TO A 128X128 ARRAY
                data_sq = np.zeros(128)
                data,sr = librosa.load(inf_path+f,sr = 16000)
                data = rescale(audio=data)
                data = librosa.util.fix_length(data,size=int(128*128))
                for index in range(0,len(data),128):
                    data_row = data[index:index+128]
                    data_sq = np.vstack((data_sq,data_row))
                data_sq = data_sq[1:129]
                data_sq = np.asarray(data_sq).astype(np.float32)
                data_sq = data_sq.transpose()
                
                # CONVERT ARRAY TO TENSOR
                data_sq = np.expand_dims(data_sq, axis=0)
                inputs = torch.from_numpy(data_sq)  
                
                ############ INFERENCE SECTION ############
                inputs = inputs.to(device)
                model_out = model(inputs)
                target_out = torch.argmax(model_out, dim=1)
                class_output = target_out.detach().item()

                # For KWS words
                if class_ix == 2 and class_output == 0: class_output = class_ix
                elif class_ix == 2 and class_output == 1: class_output = 3

                # For Horse Cough
                if class_ix == 3 and class_output == 1: class_output = class_ix
                elif class_ix == 3 and class_output == 0: class_output = 2

                
                y_true_train.append(class_ix)
                y_pred_train.append(class_output)
            except Exception as e:
                print(e)

     #print('\t\t Test Acc: {:.2f}'.format(total_acc))
   

time_end = time.time()

print('Inference Finished in {:2f} seconds'.format(time_end-time_start))

Class:  0   Remaining:  127
Class:  0   Remaining:  126
Class:  0   Remaining:  125
Class:  0   Remaining:  124
Class:  0   Remaining:  123
Class:  0   Remaining:  122
Class:  0   Remaining:  121
Class:  0   Remaining:  120
Class:  0   Remaining:  119
Class:  0   Remaining:  118
Class:  0   Remaining:  117
Class:  0   Remaining:  116
Class:  0   Remaining:  115
Class:  0   Remaining:  114
Class:  0   Remaining:  113
Class:  0   Remaining:  112
Class:  0   Remaining:  111
Class:  0   Remaining:  110
Class:  0   Remaining:  109
Class:  0   Remaining:  108
Class:  0   Remaining:  107
Class:  0   Remaining:  106
Class:  0   Remaining:  105
Class:  0   Remaining:  104
Class:  0   Remaining:  103
Class:  0   Remaining:  102
Class:  0   Remaining:  101
Class:  0   Remaining:  100
Class:  0   Remaining:  99
Class:  0   Remaining:  98
Class:  0   Remaining:  97
Class:  0   Remaining:  96
Class:  0   Remaining:  95
Class:  0   Remaining:  94
Class:  0   Remaining:  93
Class:  0   Remaining:  92


In [4]:
# processed_dir = "../data/KWS_EQUINE/processed/"
# classes = ['combined','cough']

# # LOAD DATASET FILE
# data_file = torch.load(processed_dir+'/dataset2.pt')


# y_true_train = data_file[1].cpu().numpy()
# y_pred_train = []

# time_start = time.time()

# file_count = len(y_true_train)
# model.eval()
# with torch.no_grad():
#     for counter,val in enumerate((data_file[0])):
#         val = val.numpy().astype(np.float32)
#         val = np.expand_dims(val,axis=0)
#         inputs = torch.from_numpy(val)  

#         ############ INFERENCE SECTION ############
#         inputs = inputs.to(device)
#         model_out = model(inputs)
#         target_out = torch.argmax(model_out, dim=1)
        
#         class_output = target_out.detach().item()

#         print('Remaining: ',(file_count-counter),'\tTrue:', y_true_train[counter],'\tOutput:', class_output)
#         y_pred_train.append(class_output)

#      #print('\t\t Test Acc: {:.2f}'.format(total_acc))
   

# time_end = time.time()

# print('Inference Finished in {:2f} seconds'.format(time_end-time_start))

In [5]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support

def plot_confusion(y_true, y_pred, classes):
    cf_matrix = confusion_matrix(y_true = y_true, y_pred = y_pred, labels =list(range(len(classes))))
    print(cf_matrix)

# PLOT CONFUSION MATRIX AND STAT MEASURES ON TRAIN
conf_mat_train = confusion_matrix(y_true_train, y_pred_train)
cm_display_train = ConfusionMatrixDisplay(confusion_matrix = conf_mat_train, display_labels = classes)
p_train,r_train,f1_train,_= precision_recall_fscore_support(y_true_train, y_pred_train, average=None)
cm_display_train.plot(cmap= 'Blues',colorbar=False, values_format = 'd')
plt.title('Preicison: ({:.2f} {:.2f})   Recall: ({:.2f} {:.2f})   F1-Score: ({:.2f} {:.2f})'.format(p_train[0],p_train[1],r_train[0],r_train[1],f1_train[0],f1_train[1]))

fname = model_name+'_'+str(classes[0])+'_'+str(classes[1])+'_cm_inference.png'
fname = 'test.png'
plt.savefig(checkpoint_dir+fname)
plt.clf()
plt.cla()
plt.close()